**1. Introducción**

*Nombre del proyecto:* Business Intelligence Assistant para estudio de fitness en CDMX

*Presentación del problema a abordar:*

Un estudio de fitness en la Ciudad de México requiere una herramienta de business intelligence que les permita analizar el desempeño del negocio de forma sencilla, rápida y eficaz. El estudio busca una solución cuya curva de aprendizaje no requiera de un tiempo excesivo, que permita una interacción fluida y natural, y que no implique incurrir en gastos muy elevados.

*Desarrollo de la propuesta de solución:*

Mi objetivo es desarrollar un asistente inteligente que pueda analizar los datos recolectados por el estudio, encontrar correlaciones entre los data points, identificar tendencias, graficar los resultados, y hacer recomendaciones alineadas a los KPIs del estudio.

*Justificación de la viabilidad del proyecto:*

El proyecto está siendo desarrollado a la medida de acuerdo a las especificaciones del estudio, por lo que no se requiere desarrollar una solución integral que aborde más variables que las presentadas por el estudio. El carácter limitado de dichas variables facilita el desarrollo completo de la solución, de inicio a fin, en el tiempo establecido.

**2. Objetivos**

El objetivo central es desarrollar un asistente de negocios que permita una interacción fluida con el usuario para realizar un análisis detallado del desempeño del estudio. Este asistente debe poder:

- Generar un análisis profundo de los datos de consumo de los clientes
- Convertir el análisis en gráficas que identifiquen tendencias de consumo y tendencias ocultas
- Convertir la correlación de datos en sugerencias de negocio específicas para el estudio

**3. Metodología**

Para llevar a cabo el desarrollo del asistente de negocios, los pasos a seguir son:

- Recopilación y Limpieza de Datos: Obtener información sobre la empresa (descripción, misión, visión, empleados); obtener, revisar y limpiar los datos del negocio.
- Creación de Parámetros: Utilizar GPT-4 para identificar los medidores que serán necesarios para realizar el análisis de datos, definir los cursos de acción predeterminada que se le ofrecerán al usuario, y generar un machote básico de presentación de datos para ingesta.
- Procesamiento de Texto: Utilizar GPT-4 para generar un documento que sirva como fuente de información única sobre la empresa y la marca.
- Procesamiento de Datos: Utilizar GPT-4 para analizar los datos y generar un resumen que presente tendencias de consumo y tendencias ocultas.
- Generación de Gráficas: Utilizar GPT-4 para generar gráficas como mapas de calor, gráficas de barras, gráficas de pastel, gráficas de línea de tiempo, etc.
- Generación de Recomendaciones de Negocio: Utilizar GPT-4 para generar recomendaciones personalizadas de negocio (como promociones y gestión de horarios de clases) de acuerdo a la marca y la temporada.

*Estatus del proyecto*

Actualmente, el asistente cuenta con la capacidad de recibir un archivo .csv, analizar los datos, y responder preguntas sobre los mismos.

*Siguientes pasos*

- Habilitar la capacidad de graficar los datos
- Perfeccionar el prompt en formato .txt para que el asistente tenga la capacidad de entender mejor su objetivo, alcance y limitaciones
- Agregar al prompt en formato .txt la información sobre el estudio (misión, visión, valores, servicios, KPIs, etc.)

**4. Herramientas y tecnologías**

El asistente, al tener una función específica y limitada al estudio, realizará few-shot prompting con el usuario, quien le ofrecerá un archivo que cumplirá la función del contexto. El asistente empleará dicho contexto para desarrollar sus respuestas.

*Características del asistente en su versión actual*

Versión de Python: Python 3.10.15

Librerías específicas necesarias para el desarrollo del asistente:

- Pandas y NumPy: Para la manipulación de datos, como el cálculo de los ingresos por tipo de afiliación, las cancelaciones de clases y el remuestreo de datos para el análisis de series temporales.
- Matplotlib y Seaborn: Para visualizar tendencias de cancelaciones/no-show por franja horaria, patrones de ingresos y tasas de ocupación de clases.
- Statsmodels: Para realizar análisis de regresión, en particular sobre las sanciones por no presentarse y la recuperación de la asistencia.
- Scikit-Learn: Para la agrupación y segmentación con el fin de identificar a los clientes de alto valor en función del tipo de afiliación y los patrones de asistencia.

**5. Implementación:**

*Nota: En el repositorio pueden encontrarse un archivo .txt y un .csv, usarlos para probar el código*

In [ ]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = "your_api_key"

# Function to dynamically set the context
def get_context():
    return "Eres un asistente personalizado para C Fitness Studio en la Ciudad de México, tu objetivo es analizar el negocio y proporcionar insights basados en datos."

# Load the custom prompt from a .txt file
def load_prompt(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred while loading the prompt: {e}")
        return None

# Function to upload and read a CSV file using manual input
def upload_csv_file():
    filepath = input("Enter the path to your CSV file: ").strip()
    try:
        data = pd.read_csv(filepath)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

# API call to OpenAI with custom prompt and data analysis
def ask_openai(data, messages):
    # Convert the DataFrame to a string format for the context
    data_sample = data.head().to_string()  # Include the first few rows for context
    
    # Add the data context to the messages only at the start of the chat
    if len(messages) == 1:
        initial_context = f"""
        Los siguientes son datos de desempeño del negocio en formato de tabla:

        {data_sample}

        Por favor, analiza estos datos y responde las preguntas de manera interactiva. 
        """
        messages.append({"role": "assistant", "content": initial_context})
    
    # Make the API call to OpenAI
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        max_tokens=500,
        messages=messages
    )
    
    # Append the assistant's response to the conversation
    assistant_message = response.choices[0]['message']['content']
    messages.append({"role": "assistant", "content": assistant_message})
    return assistant_message

# Main interactive chat function
def interactive_chat(data):
    print("You can now ask questions about your data. Type 'exit' to end the chat.")
    messages = [{"role": "system", "content": get_context()}]
    
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == "exit":
            print("Exiting the chat. Goodbye!")
            break
        
        # Add the user's input to the conversation history
        messages.append({"role": "user", "content": user_input})
        
        # Get the response from OpenAI
        try:
            response = ask_openai(data, messages)
            print(f"\nBot: {response}")
        except Exception as e:
            print(f"An error occurred: {e}")

# Main execution flow
# Filepath to your custom prompt
prompt_path = 'your_filepath'
custom_prompt = load_prompt(prompt_path)

if custom_prompt is not None:
    print("Custom prompt loaded successfully.")
    
    # Prompt user to upload a CSV file manually
    print("Please provide the path to your CSV file:")
    csv_data = upload_csv_file()

    if csv_data is not None:
        print("File uploaded successfully! Here's a preview of your data:")
        print(csv_data.head())
        
        # Start the interactive chat
        interactive_chat(csv_data)
    else:
        print("Failed to load and process the CSV file.")
else:
    print("Failed to load the custom prompt.")
